In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder

In [3]:
# L'URL que vous scrapez
url_ligue1 = "https://fbref.com/en/comps/13/Ligue-1-Stats"
left_on = ["Time", "Comp", "Round", "Day", "Venue", "Result", "GF", "GA", "Opponent", "Poss"]
nb_saisons = 6

# En-tête de l'utilisateur pour ressembler à Mozilla Firefox
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:97.0) Gecko/20100101 Firefox/97.0'
}

# Fonction de contrôle du taux de requêtes
MIN_REQUEST_INTERVAL = 2.5  # Interval minimum entre les requêtes (en secondes)
last_request_time = None

def rate_limit():
    global last_request_time
    if last_request_time is not None:
        elapsed_time = time.time() - last_request_time
        if elapsed_time < MIN_REQUEST_INTERVAL:
            time.sleep(MIN_REQUEST_INTERVAL - elapsed_time)
    last_request_time = time.time()

df_final = []

for i in range(nb_saisons):
    rate_limit()  # Applique le contrôle du taux de requêtes

    request1 = requests.get(url_ligue1, headers=headers)
    soup = BeautifulSoup(request1.text, 'html.parser')
    url_equipes = soup.select("table.stats_table")[0].find_all("a")
    url_equipes = [equipes.get("href") for equipes in url_equipes]
    url_equipes = [equipes for equipes in url_equipes if equipes and "squads" in equipes]
    url_equipes = [f"https://fbref.com{i}" for i in url_equipes]
    url_ligue1 = f"https://fbref.com{soup.find('a', class_='button2 prev').get('href')}"

    for j in range(len(url_equipes)):
        rate_limit()  # Applique le contrôle du taux de requêtes

        request2 = requests.get(url_equipes[j], headers=headers)
        soup2 = BeautifulSoup(request2.text, 'html.parser')
        df_equipe = pd.read_html(request2.text, match="Scores")[0]

        nom_equipe = url_equipes[j].split("/")[-1].replace("-Stats", "").replace("-", " ")
        df_equipe["equipe"] = nom_equipe

        url_stats = soup2.findAll("a")
        url_stats = [el.get("href") for el in url_stats]
        url_stats = [el for el in url_stats if el and "matchlogs/all_comps" in el]
        url_stats = [el for el in url_stats if el and ("passing/" in el or "shooting" in el or "possession/" in el or "defense/" in el or "keeper" in el)]
        url_stats = list(set(url_stats))
        url_stats = [f"https://fbref.com{i}" for i in url_stats]


        for y in range(len(url_stats)):
            rate_limit()  # Applique le contrôle du taux de requêtes

            request3 = requests.get(url_stats[y], headers=headers)
            soup3 = BeautifulSoup(request3.text, 'html.parser')
            stats = pd.read_html(request3.text)[0]

            if stats.columns.nlevels > 1:
                stats.columns = [f"{col}_{branch}" if "For" not in col and "Unnamed:" not in col else f"{branch}" for col, branch in stats.columns]

            stats.drop(left_on + [col for col in stats.columns if 'Report' in col], axis=1, inplace=True)

            df_equipe = df_equipe.merge(stats, on="Date")

        df_final.append(df_equipe)

df_final2 = pd.concat(df_final)


IndexError: list index out of range

In [29]:
url_ligue1 = "https://fbref.com/en/comps/13/Ligue-1-Stats"
left_on = ["Time", "Comp", "Round", "Day", "Venue", "Result", "GF", "GA", "Opponent"]
df_final = []

# En-tête de l'utilisateur pour ressembler à Mozilla Firefox
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:97.0) Gecko/20100101 Firefox/97.0'
}

# Fonction de contrôle du taux de requêtes
MIN_REQUEST_INTERVAL = 2.5  # Interval minimum entre les requêtes (en secondes)
last_request_time = None

def rate_limit():
    global last_request_time
    if last_request_time is not None:
        elapsed_time = time.time() - last_request_time
        if elapsed_time < MIN_REQUEST_INTERVAL:
            time.sleep(MIN_REQUEST_INTERVAL - elapsed_time)
    last_request_time = time.time()

request1 = requests.get(url_ligue1, headers=headers)
soup = BeautifulSoup(request1.text, 'html.parser')
url_equipes = soup.select("table.stats_table")[0].find_all("a")
url_equipes = [equipes.get("href") for equipes in url_equipes]
url_equipes = [equipes for equipes in url_equipes if equipes and "squads" in equipes]
url_equipes = [f"https://fbref.com{i}" for i in url_equipes]



for j in range(len(url_equipes)):
    rate_limit()  # Applique le contrôle du taux de requêtes

    request2 = requests.get(url_equipes[j], headers=headers)
    soup2 = BeautifulSoup(request2.text, 'html.parser')
    df_equipe = pd.read_html(request2.text, match="Scores")[0]

    nom_equipe = url_equipes[j].split("/")[-1].replace("-Stats", "").replace("-", " ")
    df_equipe["equipe"] = nom_equipe

    url_stats = soup2.findAll("a")
    url_stats = [el.get("href") for el in url_stats]
    url_stats = [el for el in url_stats if el and "matchlogs/all_comps" in el]
    url_stats = [el for el in url_stats if el and ("passing/" in el or "shooting" in el or "possession/" in el or "defense/" in el or "keeper" in el)]
    url_stats = list(set(url_stats))
    url_stats = [f"https://fbref.com{i}" for i in url_stats]


    for y in range(len(url_stats)):
        rate_limit()  # Applique le contrôle du taux de requêtes

        request3 = requests.get(url_stats[y], headers=headers)
        soup3 = BeautifulSoup(request3.text, 'html.parser')
        stats = pd.read_html(request3.text)[0]

        if stats.columns.nlevels > 1:
            stats.columns = [f"{col}_{branch}" if "For" not in col and "Unnamed:" not in col else f"{branch}" for col, branch in stats.columns]

        stats.drop(left_on + [col for col in stats.columns if 'Report' in col], axis=1, inplace=True)

        df_equipe = df_equipe.merge(stats, on="Date")

    df_final.append(df_equipe)

df_final2 = pd.concat(df_final)


IndexError: list index out of range